In [1]:
import pandas as pd
import numpy as np
import datetime
import sklearn

## Lectura de datos

### Datos originales

In [2]:
# Postulantes
postulantes_educacion = pd.read_csv(\
    './data/fiuba_1_postulantes_educacion.csv', low_memory=False)
postulantes_sexo_y_edad = pd.read_csv(\
    './data/fiuba_2_postulantes_genero_y_edad.csv', low_memory=False)

# Avisos
avisos_online = pd.read_csv(\
    './data/fiuba_5_avisos_online.csv', low_memory=False)
avisos_detalle = pd.read_csv(\
    './data/fiuba_6_avisos_detalle.csv', low_memory=False)

# Transacciones
vistas = pd.read_csv(\
    './data/fiuba_3_vistas.csv', low_memory=False)
postulaciones = pd.read_csv(\
    './data/fiuba_4_postulaciones.csv', low_memory=False)

### Datos hasta 15 de abril

In [3]:
# Postulantes
h15_postulantes_educacion = pd.read_csv(\
    './data/fiuba_hasta_15_abril/entrega6/fiuba_1_postulantes_educacion.csv',\
    low_memory=False)
h15_postulantes_sexo_y_edad = pd.read_csv(\
    './data/fiuba_hasta_15_abril/entrega6/fiuba_2_postulantes_genero_y_edad.csv',\
    low_memory=False)

# Avisos
h15_avisos_online = pd.read_csv(\
    './data/fiuba_hasta_15_abril/entrega6/fiuba_5_avisos_online.csv',\
    low_memory=False)
h15_avisos_detalle = pd.read_csv(\
    './data/fiuba_hasta_15_abril/entrega6/fiuba_6_avisos_detalle.csv',\
    low_memory=False)

# Transacciones
h15_vistas = pd.read_csv(\
    './data/fiuba_hasta_15_abril/entrega6/fiuba_3_vistas.csv',\
    low_memory=False)
h15_postulaciones = pd.read_csv(\
    './data/fiuba_hasta_15_abril/entrega6/fiuba_4_postulaciones.csv',\
    low_memory=False)

### Datos desde el 15 de abril

In [4]:
# Postulantes
d15_postulantes_educacion = pd.read_csv(\
    './data/fiuba_desde_15_Abril/fiuba_1_postulantes_educacion.csv',\
    low_memory=False)
d15_postulantes_sexo_y_edad = pd.read_csv(\
    './data/fiuba_desde_15_Abril/fiuba_2_postulantes_genero_y_edad.csv',\
    low_memory=False)

# Avisos
d15_avisos_detalle = pd.read_csv(\
    './data/fiuba_desde_15_Abril/fiuba_6_avisos_detalle.csv',\
    low_memory=False)

# Transacciones
d15_vistas = pd.read_csv(\
    './data/fiuba_desde_15_Abril/fiuba_3_vistas.csv',\
    low_memory=False)

### Detalle de avisos faltantes

In [5]:
# Avisos
m_avisos_detalle = pd.read_csv(\
    './data/fiuba_6_avisos_detalle_missing_nivel_laboral.csv',\
    low_memory=False)

## Merge de todos los datos

### Merge de los postulantes

Agregamos columna con ponderación del nivel de estudios alcanzado.

In [6]:
def categorizar_estudios(x):
    if((x is np.nan) | (x == 'Otro')):
        return 0
    if(x == 'Secundario'):
        return 1
    if(x == 'Terciario/Técnico'):
        return 2
    if(x == 'Universitario'):
        return 3
    if((x == 'Posgrado') | (x == 'Master') | (x == 'Doctorado')):
        return 4

postulantes_educacion['nivel_alcanzado'] = postulantes_educacion['nombre']\
    .apply(lambda x: categorizar_estudios(x))
h15_postulantes_educacion['nivel_alcanzado'] = h15_postulantes_educacion['nombre']\
    .apply(lambda x: categorizar_estudios(x))
d15_postulantes_educacion['nivel_alcanzado'] = d15_postulantes_educacion['nombre']\
    .apply(lambda x: categorizar_estudios(x))
#postulantes_educacion.head()

Unimos todos los dataframes de la educación de los postulantes y nos quedamos solo con los registros del máximo nivel de estudios alcanzado.

In [7]:
m_postulantes_educacion = pd.concat([postulantes_educacion, \
                                 h15_postulantes_educacion, \
                                 d15_postulantes_educacion])

In [8]:
m_postulantes_educacion = m_postulantes_educacion \
    .sort_values('nivel_alcanzado', ascending = False) \
    .drop_duplicates('idpostulante')

In [9]:
del m_postulantes_educacion['nombre']
m_postulantes_educacion.head(2)

,idpostulante,estado,nivel_alcanzado
0,NdJl,En Curso,4
237833,e4oaA2,En Curso,4


Unimos (concat) todos los dataframes de sexo y edad de los postulantes y luego los unimos (outer join) con los dataframes de los niveles de educación de los postulantes previamente tratados.

In [10]:
m_postulantes_sexo_y_edad = pd.concat([postulantes_sexo_y_edad, \
                                      h15_postulantes_sexo_y_edad, \
                                      d15_postulantes_sexo_y_edad])

In [11]:
m_postulantes_sexo_y_edad.count()

idpostulante       780020
fechanacimiento    745747
sexo               780020
dtype: int64

In [12]:
m_postulantes_sexo_y_edad = m_postulantes_sexo_y_edad.drop_duplicates()
m_postulantes_sexo_y_edad.count()

idpostulante       505382
fechanacimiento    478699
sexo               505382
dtype: int64

In [12]:
#m_postulantes_sexo_y_edad = m_postulantes_sexo_y_edad.groupby(['idpostulante'])\
#    .apply(lambda x: x.loc[x.sexo != 'NO_DECLARA',:] if len(x)>1 else x)\
#    .reset_index(drop=True)

In [13]:
m_postulantes_sexo_y_edad.count()

idpostulante       505382
fechanacimiento    478699
sexo               505382
dtype: int64

In [14]:
m_postulantes_sexo_y_edad = m_postulantes_sexo_y_edad.drop_duplicates('idpostulante')
m_postulantes_sexo_y_edad.count()

idpostulante       504407
fechanacimiento    477724
sexo               504407
dtype: int64

In [15]:
m_postulantes_sexo_y_edad['fechanacimiento'] = \
    pd.to_datetime(postulantes_sexo_y_edad['fechanacimiento'], errors='coerce')
m_postulantes_sexo_y_edad.count()

idpostulante       504407
fechanacimiento    384151
sexo               504407
dtype: int64

In [16]:
def calcular_edad(fecha_nacimiento):
    hoy = datetime.date.today()
    edad = hoy.year - fecha_nacimiento.year - ((hoy.month, hoy.day)\
            < (fecha_nacimiento.month, fecha_nacimiento.day))
    return np.NaN if ((edad > 100) | (edad < 16)) else edad

m_postulantes_sexo_y_edad['edad'] = \
    m_postulantes_sexo_y_edad['fechanacimiento']\
    .apply(lambda x: calcular_edad(x))
m_postulantes_sexo_y_edad.head()

,idpostulante,fechanacimiento,sexo,edad
0,NM5M,1970-12-03,FEM,47.0
1,5awk,1962-12-04,FEM,55.0
2,ZaO5,1978-08-10,FEM,39.0
3,NdJl,1969-05-09,MASC,49.0
4,eo2p,1981-02-16,MASC,37.0


In [17]:
edad_promedio = m_postulantes_sexo_y_edad['edad'].mean()
m_postulantes_sexo_y_edad['edad'].fillna(edad_promedio, inplace = True)
m_postulantes_sexo_y_edad.drop(['fechanacimiento'], axis=1, inplace=True)
m_postulantes_sexo_y_edad.head(2)

,idpostulante,sexo,edad
0,NM5M,FEM,47.0
1,5awk,FEM,55.0


In [18]:
m_postulantes = pd.merge(m_postulantes_sexo_y_edad, m_postulantes_educacion, \
                       on = 'idpostulante', how='outer')
m_postulantes.count()

idpostulante       504407
sexo               504407
edad               504407
estado             447909
nivel_alcanzado    447909
dtype: int64

In [19]:
nivel_promedio = int(m_postulantes.nivel_alcanzado.mean())
m_postulantes['nivel_alcanzado'].fillna(nivel_promedio, inplace = True)
m_postulantes.count()

idpostulante       504407
sexo               504407
edad               504407
estado             447909
nivel_alcanzado    504407
dtype: int64

In [20]:
m_postulantes.estado.value_counts()

Graduado      253833
En Curso      148072
Abandonado     46004
Name: estado, dtype: int64

In [21]:
def categorizar_estados(estado):
    if (estado == 'Graduado'):
        return 2
    if (estado == 'En Curso'):
        return 1
    if (estado == 'Abandonado'):
        return 0
m_postulantes['estado'] = m_postulantes['estado'].apply(lambda x : categorizar_estados(x))
m_postulantes.count()

idpostulante       504407
sexo               504407
edad               504407
estado             447909
nivel_alcanzado    504407
dtype: int64

In [22]:
estado_promedio = m_postulantes.estado.median()
m_postulantes['estado'].fillna(estado_promedio, inplace = True)
m_postulantes.count()

idpostulante       504407
sexo               504407
edad               504407
estado             504407
nivel_alcanzado    504407
dtype: int64

In [23]:
m_postulantes.head(2)

,idpostulante,sexo,edad,estado,nivel_alcanzado
0,NM5M,FEM,47.0,2.0,1.0
1,5awk,FEM,55.0,2.0,3.0


### Merge de los avisos

In [24]:
avisos_detalle = avisos_detalle.loc[:,['idaviso', 'titulo', \
    'nombre_zona', 'tipo_de_trabajo', 'nivel_laboral', 'nombre_area',\
    'denominacion_empresa']]
h15_avisos_detalle = h15_avisos_detalle.loc[:,['idaviso', 'titulo', \
    'nombre_zona', 'tipo_de_trabajo', 'nivel_laboral', 'nombre_area',\
    'denominacion_empresa']]
d15_avisos_detalle = d15_avisos_detalle.loc[:,['idaviso', 'titulo', \
    'nombre_zona', 'tipo_de_trabajo', 'nivel_laboral', 'nombre_area',\
    'denominacion_empresa']]
m_avisos_detalle = m_avisos_detalle.loc[:,['idaviso', 'titulo', \
    'nombre_zona', 'tipo_de_trabajo', 'nivel_laboral', 'nombre_area',\
    'denominacion_empresa']]

In [25]:
m_avisos = pd.concat([avisos_detalle, h15_avisos_detalle, \
                      d15_avisos_detalle, m_avisos_detalle])

In [26]:
m_avisos.count()

idaviso                 45969
titulo                  45969
nombre_zona             45969
tipo_de_trabajo         45969
nivel_laboral           45634
nombre_area             45969
denominacion_empresa    45955
dtype: int64

In [27]:
m_avisos = m_avisos.drop_duplicates()
m_avisos.count()

idaviso                 25637
titulo                  25637
nombre_zona             25637
tipo_de_trabajo         25637
nivel_laboral           25302
nombre_area             25637
denominacion_empresa    25630
dtype: int64

In [28]:
m_avisos = m_avisos.drop_duplicates('idaviso')
m_avisos.count()

idaviso                 25288
titulo                  25288
nombre_zona             25288
tipo_de_trabajo         25288
nivel_laboral           24953
nombre_area             25288
denominacion_empresa    25281
dtype: int64

In [29]:
m_avisos.head(2)

,idaviso,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,VENDEDOR/A PROVINCIA DE SANTA FE,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Comercial,VENTOR
1,17903700,Enfermeras,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Salud,Farmacias Central Oeste


In [30]:
m_avisos.nivel_laboral.value_counts()

Senior / Semi-Senior                    16983
Junior                                   4149
Otro                                     1972
Jefe / Supervisor / Responsable          1527
Gerencia / Alta Gerencia / Dirección      322
Name: nivel_laboral, dtype: int64

In [31]:
# Completo los NaNs del nivel laboral con "Senior / Semi-Senior"
m_avisos['nivel_laboral'].fillna('Senior / Semi-Senior', inplace = True)
m_avisos.count()

idaviso                 25288
titulo                  25288
nombre_zona             25288
tipo_de_trabajo         25288
nivel_laboral           25288
nombre_area             25288
denominacion_empresa    25281
dtype: int64

In [32]:
def categorizar_nivel_laboral(x):
    if(x == 'Junior'):
        return 0
    if(x == 'Senior / Semi-Senior'):
        return 1
    if((x == 'Jefe / Supervisor / Responsable') |
      (x == 'Gerencia / Alta Gerencia / Dirección')):
        return 2
    return 3

m_avisos['nivel_laboral'] = m_avisos['nivel_laboral']\
    .apply(lambda x: categorizar_nivel_laboral(x))

In [33]:
# Completo los NaNs del nombre de la empresa con "No declara"
m_avisos['denominacion_empresa'].fillna('No declara', inplace = True)
m_avisos.count()

idaviso                 25288
titulo                  25288
nombre_zona             25288
tipo_de_trabajo         25288
nivel_laboral           25288
nombre_area             25288
denominacion_empresa    25288
dtype: int64

In [34]:
m_avisos.head(2)

,idaviso,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,VENDEDOR/A PROVINCIA DE SANTA FE,Gran Buenos Aires,Full-time,1,Comercial,VENTOR
1,17903700,Enfermeras,Gran Buenos Aires,Full-time,1,Salud,Farmacias Central Oeste


In [35]:
m_avisos.tipo_de_trabajo.value_counts()

Full-time          22830
Part-time           1746
Teletrabajo          248
Por Horas            127
Pasantia             119
Temporario            95
Por Contrato          88
Fines de Semana       28
Primer empleo          6
Voluntario             1
Name: tipo_de_trabajo, dtype: int64

In [36]:
def categorizar_tipo_de_trabajo(x):
    if(x == 'Full-time'):
        return 1
    return 0

m_avisos['tipo_de_trabajo'] = m_avisos['tipo_de_trabajo']\
    .apply(lambda x: categorizar_tipo_de_trabajo(x))

In [37]:
type(m_avisos.nombre_area)

pandas.core.series.Series

In [48]:
def categorizar_area(x):
    if ((type(x) is str) == False):
        return 'Otros'
    if ((x == 'Programación') | (x == 'Soporte Técnico') |
        (x == 'Tecnologia / Sistemas') |
        (x == 'Sistemas') | (x == 'Redes') |
        (x == 'Business Intelligence') | (x == 'Telecomunicaciones') |
        (x == 'Tecnologías de la Información') |
        (x == 'Seguridad Informática') | (x == 'Análisis Funcional') |
        (x == 'Internet') | (x == 'Diseño Web') |
        (x == 'Diseño Multimedia') | (x == 'Testing / QA / QC') |
        (x == 'Data Warehousing') | (x == 'Multimedia') |
        (x == 'Administración de Base de Datos')):
        return 'Sistemas'
    if ((x == 'Ventas') | (x == 'Comercial') | (x == 'Compras') |
        (x == 'Administración') | (x == 'Contabilidad') |
        (x == 'Marketing') | (x == 'Finanzas') | (x == 'Tesorería') |
        (x == 'Créditos y Cobranzas') | (x == 'Legal') |
        (x == 'Desarrollo de Negocios') | (x == 'Impuestos') |
        (x == 'Auditoría') | (x == 'Control de Gestión') |
        (x == 'Planeamiento comercial') | (x == 'Facturación') |
        (x == 'Compras Internacionales/Importación') |
        (x == 'Consultorías Comercio Exterior') | (x == 'Consultoria') |
        (x == 'Análisis de Riesgos') | (x == 'E-commerce') |
        (x == 'Ventas Internacionales/Exportación') |
        (x == 'Comercio Exterior') | (x == 'Evaluación Económica') |
        (x == 'Mercadotecnia Internacional') | 
        (x == 'Relaciones Institucionales/Publicas') |
        (x == 'Planeamiento económico-financiero ') |
        (x == 'Corporate Finance / Banca Inversión') |
        (x == 'Cuentas Corrientes')):
        return 'Comercial / Negocios'
    if ((x == 'Salud') | (x == 'Medicina') | (x == 'Farmacéutica') |
        (x == 'Medicina Laboral') | (x == 'Otras áreas técnicas en salud') |
        (x == 'Fonoaudiología') | (x == 'Farmacia hospitalaria') |
        (x == 'Prácticas cardiológicas') | (x == 'Odontología') |
        (x == 'Otras Especialidades médicas') | (x == 'Clínica Médica') |
        (x == 'Kinesiología / terapia ocupacional') |
        (x == 'Auditoría Médica') | (x == 'Instrumentación quirúrgica') |
        (x == 'Enfermería (ver Enfermería)') | (x == 'Psicología')):
        return 'Salud'
    if ((x == 'Producción') | (x == 'Logística') | (x == 'Construcción') |
        (x == 'Ingeniería  Industrial') | (x == 'Ingeniería  Mecánica') |
        (x == 'Ingeniería  Eléctrica y Electrónica ') |
        (x == 'Distribución') | (x == 'Programación de producción') |
        (x == 'Producto') | (x == 'Otras Ingenierias') | 
        (x == 'Mineria/Petroleo/Gas') | (x == 'Ingeniería Civil') |
        (x == 'Ingeniería Oficina Técnica / Proyecto') |
        (x == 'Dirección de Obra') | (x == 'Ingeniería Química') |
        (x == 'Ingeniería Electromecánica') | (x == 'Seguridad e Higiene') |
        (x == 'Ingeniería de Procesos') | (x == 'Organización y Métodos') |
        (x == 'Ingeniería en Alimentos') | (x == 'Ingeniería  Automotriz') |
        (x == 'Ingeniería Agrónoma') | (x == 'Operaciones') |
        (x == 'Ingeniería de Producto ') | (x == 'Ingeniería  Metalurgica') |
        (x == 'Planeamiento') | (x == 'Farmacia industrial') |
        (x == 'Seguridad Industrial') | (x == 'Quimica') |
        (x == 'Química') | (x == 'Ingeniería en Petróleo y Petroquímica') |
        (x == 'Ingeniería Geológica') | (x == 'Instrumentación Minera') |
        (x == 'Exploración Minera y Petroquimica') | (x == 'Infraestructura') |
        (x == 'Ingeniería en Minas') | (x == 'Calidad') |
        (x == 'Abastecimiento') | (x == 'Ingeniería de Ventas ') |
        (x == 'Diseño Industrial')):
        return 'Produccion / Ingenieria'
    if ((x == 'Atención al Cliente') | (x == 'Mantenimiento') |
        (x == 'Mantenimiento y Limpieza') | (x == 'Call Center') |
        (x == 'Recepcionista') | (x == 'Oficios y Profesiones') |
        (x == 'Almacén / Depósito / Expedición') | (x == 'Telemarketing') |
        (x == 'Seguridad') | (x == 'Secretaria') | (x == 'Camareros') |
        (x == 'Cadetería') | (x == 'Asistente') | (x == 'Data Entry') |
        (x == 'Pasantía / Trainee') | (x == 'Caja') | (x == 'Back Office') |
        (x == 'Promotoras/es') | (x == 'Telefonista')):
        return 'No calificado'
    if ((x == 'Recursos Humanos') | (x == 'Selección') |
        (x == 'Administración de Personal') |
        (x == 'Compensación y Planilla')):
        return 'RRHH'
    if ((x == 'Turismo') | (x == 'Hotelería')):
        return 'Turismo'
    if ((x == 'Educación/ Docentes') | (x == 'Educación') |
        (x == 'Educación especial') | (x == 'Capacitación') |
        (x == 'Bienestar Estudiantil')):
        return 'Educacion'
    if ((x == 'Arquitectura') | (x == 'Diseño Gráfico') |
        (x == 'Diseño Textil e Indumentaria') | (x == 'Diseño') |
        (x == 'Diseño de Interiores / Decoración') | (x == 'Urbanismo') |
        (x == 'Diseño 3D')):
        return 'Disenio'
    return 'Otros'

m_avisos['nombre_area'] = m_avisos['nombre_area']\
    .apply(lambda x: categorizar_area(x))
m_avisos.nombre_area.value_counts()

Comercial / Negocios       10350
No calificado               3936
Produccion / Ingenieria     3930
Sistemas                    2938
Otros                       2099
RRHH                         944
Salud                        598
Disenio                      242
Educacion                    139
Turismo                      112
Name: nombre_area, dtype: int64

In [49]:
m_avisos.head()

,idaviso,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,VENDEDOR/A PROVINCIA DE SANTA FE,Gran Buenos Aires,1,1,Comercial / Negocios,VENTOR
1,17903700,Enfermeras,Gran Buenos Aires,1,1,Salud,Farmacias Central Oeste
2,1000150677,Chofer de taxi,Capital Federal,1,1,Otros,FAMITAX SRL
3,1000610287,CHOFER DE CAMIONETA BAHIA BLANCA - PUNTA ALTA,Gran Buenos Aires,1,1,Otros,Wurth Argentina S.A
4,1000872556,Operarios de Planta - Rubro Electrodomésticos,Gran Buenos Aires,1,1,Produccion / Ingenieria,ELECTRO OUTLET SRL


In [50]:
del m_avisos['titulo']

In [52]:
m_avisos.nombre_zona.value_counts()

Gran Buenos Aires              23015
Capital Federal                 2185
Buenos Aires (fuera de GBA)       57
GBA Oeste                         10
La Plata                           4
Mendoza                            3
Ciudad de Mendoza                  3
Cordoba                            2
Rosario                            2
Catamarca                          1
Corrientes                         1
Tucuman                            1
Santa Fe                           1
San Juan                           1
Neuquen                            1
Santa Cruz                         1
Name: nombre_zona, dtype: int64

In [53]:
def categorizar_zona(x):
    if (x == 'Capital Federal'):
        return 0
    if ((x == 'Gran Buenos Aires') | (x == 'GBA Oeste')):
        return 1
    return 2

m_avisos['nombre_zona'] = m_avisos['nombre_zona']\
    .apply(lambda x: categorizar_zona(x))
m_avisos.nombre_zona.value_counts()

1    23025
0     2185
2       78
Name: nombre_zona, dtype: int64

### Label encoding

In [51]:
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [ ]:
#del m_avisos['denominacion_empresa']

In [55]:
avisos_encoding = MultiColumnLabelEncoder(columns = \
    ['nombre_area','denominacion_empresa']).fit_transform(m_avisos)
avisos_encoding.head()

,idaviso,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,1,1,1,0,4005
1,17903700,1,1,1,7,1640
2,1000150677,0,1,1,4,1561
3,1000610287,1,1,1,4,4119
4,1000872556,1,1,1,5,1267


In [56]:
postulantes_encoding = MultiColumnLabelEncoder(columns = \
    ['sexo']).fit_transform(m_postulantes)
postulantes_encoding.head()

,idpostulante,sexo,edad,estado,nivel_alcanzado
0,NM5M,1,47.0,2.0,1.0
1,5awk,1,55.0,2.0,3.0
2,ZaO5,1,39.0,0.0,3.0
3,NdJl,2,49.0,1.0,4.0
4,eo2p,2,37.0,2.0,4.0


### Merge de las postulaciones

In [57]:
m_postulaciones = pd.concat([postulaciones, h15_postulaciones])

In [58]:
m_postulaciones['fechapostulacion'] = \
    pd.to_datetime(m_postulaciones['fechapostulacion'], errors='coerce')
m_postulaciones.count()

idaviso             8311264
idpostulante        8311264
fechapostulacion    8311264
dtype: int64

In [62]:
del m_postulaciones['fechapostulacion']
m_postulaciones.head(2)

,idaviso,idpostulante
0,1112257047,NM5M
1,1111920714,NM5M


In [63]:
postulantes_encoding.count()

idpostulante       504407
sexo               504407
edad               504407
estado             504407
nivel_alcanzado    504407
dtype: int64

In [64]:
merged = pd.merge(m_postulaciones, postulantes_encoding, \
                  on = 'idpostulante', how = 'inner')

In [65]:
merged.count()

idaviso            8311264
idpostulante       8311264
sexo               8311264
edad               8311264
estado             8311264
nivel_alcanzado    8311264
dtype: int64

In [66]:
avisos_encoding.count()

idaviso                 25288
nombre_zona             25288
tipo_de_trabajo         25288
nivel_laboral           25288
nombre_area             25288
denominacion_empresa    25288
dtype: int64

In [67]:
merged2 = pd.merge(merged, avisos_encoding, on = 'idaviso', how = 'inner')
merged2.count()

idaviso                 7742942
idpostulante            7742942
sexo                    7742942
edad                    7742942
estado                  7742942
nivel_alcanzado         7742942
nombre_zona             7742942
tipo_de_trabajo         7742942
nivel_laboral           7742942
nombre_area             7742942
denominacion_empresa    7742942
dtype: int64

In [68]:
merged2.head()

,idaviso,idpostulante,sexo,edad,estado,nivel_alcanzado,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1112257047,NM5M,1,47.0,2.0,1.0,1,0,3,3,1369
1,1112257047,1kJqGb,1,34.0,1.0,3.0,1,0,3,3,1369
2,1112257047,eOE9Rr,1,39.0,0.0,3.0,1,0,3,3,1369
3,1112257047,Zrx8Xz,1,39.0,2.0,2.0,1,0,3,3,1369
4,1112257047,ZrKNQY,1,32.0,1.0,3.0,1,0,3,3,1369


In [69]:
merged2.drop_duplicates()
merged2.count()

idaviso                 7742942
idpostulante            7742942
sexo                    7742942
edad                    7742942
estado                  7742942
nivel_alcanzado         7742942
nombre_zona             7742942
tipo_de_trabajo         7742942
nivel_laboral           7742942
nombre_area             7742942
denominacion_empresa    7742942
dtype: int64

In [70]:
merged2.to_csv('./data/merged_encoding_01.csv', encoding='utf-8')
avisos_encoding.to_csv('./data/avisos_encoding_01.csv', encoding='utf-8')
postulantes_encoding.to_csv('./data/postulantes_encoding_01.csv', encoding='utf-8')